In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
import os
import torch
from torch.utils.data import Dataset
from skimage import io
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
# from torchsummary import summary
from PIL import Image

In [3]:
!pip install torchsummary
from torchsummary import summary

In [3]:
X_train = pd.read_csv('/kaggle/input/col774-2022/train_x.csv')    # reading the csv file
X_train.head() 

,Id,Cover_image_name,Title
0,0,33735.jpg,Lutheran and Catholic Reconciliation on Justif...
1,1,36310.jpg,Uncertain Justice: The Roberts Court : and the...
2,2,23735.jpg,Currency ; Trading For Dummies
3,3,45580.jpg,": Twins, Triplets, and More: Their Nature, Dev..."
4,4,14290.jpg,The 2013 ; Import and Export Market for Printe...


In [4]:
y_train= pd.read_csv("/kaggle/input/col774-2022/train_y.csv")
y_test= pd.read_csv("/kaggle/input/col774-2022/non_comp_test_y.csv")

In [5]:
X_test = pd.read_csv("/kaggle/input/col774-2022/non_comp_test_x.csv")

In [7]:
y_train.head()

,Id,Genre
0,0,4
1,1,16
2,2,23
3,3,7
4,4,28


In [8]:
X_test.head(2)

,Id,Cover_image_name,Title
0,0,23538.jpg,"Venus and , Serena Williams (Great African Ame..."
1,1,11016.jpg,Faithfully Yours: The : Amazing Bond Between U...


In [6]:
train = pd.merge(X_train, y_train, on='Id')

In [10]:
train.head()

,Id,Cover_image_name,Title,Genre
0,0,33735.jpg,Lutheran and Catholic Reconciliation on Justif...,4
1,1,36310.jpg,Uncertain Justice: The Roberts Court : and the...,16
2,2,23735.jpg,Currency ; Trading For Dummies,23
3,3,45580.jpg,": Twins, Triplets, and More: Their Nature, Dev...",7
4,4,14290.jpg,The 2013 ; Import and Export Market for Printe...,28


In [7]:
test= pd.merge(X_test, y_test, on='Id')
test.head()

,Id,Cover_image_name,Title,Genre
0,0,23538.jpg,"Venus and , Serena Williams (Great African Ame...",25
1,1,11016.jpg,Faithfully Yours: The : Amazing Bond Between U...,1
2,2,485.jpg,The China Mirage: The : Hidden History of Amer...,13
3,3,21793.jpg,My Kind of Sad: What ; It's Like to Be Young a...,21
4,4,23195.jpg,ASE Test Preparation - T1 Gasoline Engines (As...,6


In [8]:
class genre_class(Dataset):
    def __init__(self, dataframe, root_dir, transform= None):
        self.annotations= dataframe
        self.root_dir= root_dir
        self.transform= transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path= os.path.join(self.root_dir, self.annotations.iloc[index,1])
#         image= io.imread(img_path)
        image = Image.open(img_path).convert('RGB')
        y_label= torch.tensor(int(self.annotations.iloc[index,3]))
        
        if self.transform:
            image= self.transform(image)
            
        return(image, y_label)

Hyper parameters

In [9]:
in_channel= 3
num_classes= 30
learning_rate= 0.001
batch_size = 64
num_epochs= 20
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Load data

In [10]:
train_set= genre_class(dataframe= train, root_dir='/kaggle/input/col774-2022/images/images/', 
              transform= transforms.Compose([
                  transforms.Resize((128), antialias=True),
                  transforms.ToTensor()]
              ))
train_loader= DataLoader(dataset=train_set,batch_size= batch_size, shuffle=False)

In [11]:
test= genre_class(dataframe= test, root_dir='/kaggle/input/col774-2022/images/images/', 
              transform= transforms.Compose([
                  transforms.Resize((128), antialias=True),
                  transforms.ToTensor()]
              ))
test_loader= DataLoader(dataset=test,batch_size= batch_size, shuffle=False)

In [16]:
for ele in train_loader:
    print(ele[0].shape)
    break

torch.Size([64, 3, 128, 128])


In [12]:
for ele in test_loader:
    print(ele[0].shape)
    break

torch.Size([64, 3, 128, 128])


create model

In [13]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
# Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2)
#         print(self.max_pool1.size())
        # Current Image Size = (62 X 62 X 32)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2)
        # Current Image Size = (29 X 29 X 64)
        self.conv_layer3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        self.relu3 = nn.ReLU()
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2)
#         Curremt Image Size = (12 X 12 X 128)
        self.fc1 = nn.Linear(18432, 128)
#         self.fc1 = nn.Linear(144,128)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
        
       # Progresses data across layers    
    def forward(self, x):
#         print(x.size())
        out = self.conv_layer1(x)
#         print(out.size())
        out = self.relu1(out)
#         print(out.size())
        out = self.max_pool1(out)
#         print(out.size())
        out = self.conv_layer2(out)
#         print(out.size())
        out = self.relu2(out)
#         print(out.size())
        out = self.max_pool2(out)
#         print(out.size())
        out = self.conv_layer3(out)
#         print(out.size())
        out = self.relu3(out)
#         print(out.size())
        out = self.max_pool3(out)    
#         print(out.size())
        out = out.reshape(out.size(0), -1)
#         print(out.size())
#         out = torch.flatten(out)
#         print(out.size())
        out = self.fc1(out)
#         print(out.size())
        out = self.relu4(out)
#         print(out.size())
        out = self.fc2(out)
#         print(out.size())
        return out


In [14]:
model = ConvNeuralNet(num_classes)

In [15]:
#Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  


train network 

In [25]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader): 
#         break
#     break
        # Forward pass
        try:
            images = images.to(device)
            labels = labels.to(device)
#             model= nn.DataParallel(model)
            model = model.to(device)
#             model = model.to(device)
            outputs = model(images)
    #         print(outputs)
            labels = labels.to(torch.long)
            outputs = outputs.to(torch.double)
    #         print(labels)
            outputs  = torch.reshape(outputs, (batch_size,-1))
    #         print(outputs)
            loss = criterion(outputs, labels)
#             print(i, loss.item())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        except Exception as e:
            print(e) 
            continue
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

shape '[64, -1]' is invalid for input of size 720
Epoch [1/20], Loss: 3.3033
shape '[64, -1]' is invalid for input of size 720
Epoch [2/20], Loss: 3.2935
shape '[64, -1]' is invalid for input of size 720
Epoch [3/20], Loss: 3.2713
shape '[64, -1]' is invalid for input of size 720
Epoch [4/20], Loss: 3.0921
shape '[64, -1]' is invalid for input of size 720
Epoch [5/20], Loss: 2.7443
shape '[64, -1]' is invalid for input of size 720
Epoch [6/20], Loss: 2.5028
shape '[64, -1]' is invalid for input of size 720
Epoch [7/20], Loss: 2.1902
shape '[64, -1]' is invalid for input of size 720
Epoch [8/20], Loss: 1.7527
shape '[64, -1]' is invalid for input of size 720
Epoch [9/20], Loss: 1.9730
shape '[64, -1]' is invalid for input of size 720
Epoch [10/20], Loss: 1.3831
shape '[64, -1]' is invalid for input of size 720
Epoch [11/20], Loss: 1.5040
shape '[64, -1]' is invalid for input of size 720
Epoch [12/20], Loss: 1.1769
shape '[64, -1]' is invalid for input of size 720
Epoch [13/20], Loss: 1.

In [26]:
def check_accuracy(loader, model):
    num_correct= 0
    num_samples= 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            score= model(x)
            _, predictions= score.max(1)
            num_correct += (predictions==y.sum())
            num_smaples += predictions.size()
    
    model.train()

In [29]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))

Accuracy of the network on the 50000 train images: 43.66374269005848 %


In [16]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(test_loader): 
#         break
#     break
        # Forward pass
        try:
            images = images.to(device)
            labels = labels.to(device)
#             model= nn.DataParallel(model)
            model = model.to(device)
#             model = model.to(device)
            outputs = model(images)
    #         print(outputs)
            labels = labels.to(torch.long)
            outputs = outputs.to(torch.double)
    #         print(labels)
            outputs  = torch.reshape(outputs, (batch_size,-1))
    #         print(outputs)
            loss = criterion(outputs, labels)
#             print(i, loss.item())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        except Exception as e:
            print(e) 
            continue
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

shape '[64, -1]' is invalid for input of size 120
Epoch [1/20], Loss: 3.4026
shape '[64, -1]' is invalid for input of size 120
Epoch [2/20], Loss: 3.3783
shape '[64, -1]' is invalid for input of size 120
Epoch [3/20], Loss: 3.3694
shape '[64, -1]' is invalid for input of size 120
Epoch [4/20], Loss: 3.3759
shape '[64, -1]' is invalid for input of size 120
Epoch [5/20], Loss: 3.3086
shape '[64, -1]' is invalid for input of size 120
Epoch [6/20], Loss: 3.1484
shape '[64, -1]' is invalid for input of size 120
Epoch [7/20], Loss: 3.1127
shape '[64, -1]' is invalid for input of size 120
Epoch [8/20], Loss: 2.8552
shape '[64, -1]' is invalid for input of size 120
Epoch [9/20], Loss: 2.4773
shape '[64, -1]' is invalid for input of size 120
Epoch [10/20], Loss: 2.2447
shape '[64, -1]' is invalid for input of size 120
Epoch [11/20], Loss: 1.7267
shape '[64, -1]' is invalid for input of size 120
Epoch [12/20], Loss: 1.4585
shape '[64, -1]' is invalid for input of size 120
Epoch [13/20], Loss: 1.

In [17]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the test images: {} %'.format( 100 * correct / total))

Accuracy of the network on the test images: 75.56140350877193 %
